In [1]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import PIL
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
ACTIONS = [['LEFT'], ['RIGHT'], ['LEFT', 'DOWN'], ['RIGHT', 'DOWN'], ['DOWN'],
           ['DOWN', 'B'], ['B']]

In [3]:
from visualize.visualize import gen_eval_vid_frame
from a2c_ppo_acktr.envs import make_vec_envs

from PIL import Image

In [4]:
seed = 10
device = "cuda:0"
env_state = "GreenHillZone.Act1"
env = make_vec_envs(
    [env_state],
    seed + 1000,
    1,
    None,
    device=device,
    allow_early_resets=False,
    mode='eval')

In [5]:
# model_path = "saved-runs/05-25_00-33-44/ppo/GreenHillZone.Act1/ckpts/runs_05-25_00-33-44_ppo_GreenHillZone.Act1-1000704.pt"
model_path = "runs/05-25_23-58-48/ppo/GreenHillZone.Act1/ckpts/runs_05-25_23-58-48_ppo_GreenHillZone.Act1-100608.pt"
model = torch.load(model_path)
print(model)
actor_critic = model[0]
# from a2c_ppo_acktr.model import Policy
# obs_space = env.observation_space
# obs_module = {'video': 'video-large', 'audio': 'audio-small'}
# assert set(obs_space.spaces.keys()) == set(obs_module.keys()), "Observation spaces and modules must have same keys."
# actor_critic = Policy(
#     obs_space,
#     obs_module,
#     env.action_space,
#     base_kwargs={'recurrent': False})
# env_step = 0
# episode_num = 0
actor_critic.to(device)
actor_critic.train()

[Policy(
  (base): NNBase2(
    (audio_module): Sequential(
      (0): ProcessMelSpectrogram(
        (mel_s): MelSpectrogram(
          (spectrogram): Spectrogram()
          (mel_scale): MelScale()
        )
        (a_to_db): AmplitudeToDB()
      )
      (1): Conv2d(1, 32, kernel_size=(8, 8), stride=(4, 4))
      (2): ReLU()
      (3): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2))
      (4): ReLU()
      (5): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (6): ReLU()
      (7): Flatten()
      (8): Linear(in_features=7168, out_features=256, bias=True)
      (9): ReLU()
    )
    (video_module): Sequential(
      (0): Divide()
      (1): Conv2d(3, 32, kernel_size=(8, 8), stride=(4, 4))
      (2): ReLU()
      (3): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
      (4): ReLU()
      (5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (6): ReLU()
      (7): Flatten()
      (8): Linear(in_features=55296, out_features=512, bias=True)
      (9): ReLU()
    )
  

Policy(
  (base): NNBase2(
    (audio_module): Sequential(
      (0): ProcessMelSpectrogram(
        (mel_s): MelSpectrogram(
          (spectrogram): Spectrogram()
          (mel_scale): MelScale()
        )
        (a_to_db): AmplitudeToDB()
      )
      (1): Conv2d(1, 32, kernel_size=(8, 8), stride=(4, 4))
      (2): ReLU()
      (3): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2))
      (4): ReLU()
      (5): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (6): ReLU()
      (7): Flatten()
      (8): Linear(in_features=7168, out_features=256, bias=True)
      (9): ReLU()
    )
    (video_module): Sequential(
      (0): Divide()
      (1): Conv2d(3, 32, kernel_size=(8, 8), stride=(4, 4))
      (2): ReLU()
      (3): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
      (4): ReLU()
      (5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (6): ReLU()
      (7): Flatten()
      (8): Linear(in_features=55296, out_features=512, bias=True)
      (9): ReLU()
    )
  )

In [6]:
recurrent_hidden_states = torch.zeros(1, actor_critic.recurrent_hidden_state_size).to(device)
masks = torch.zeros(1, 1).to(device)

obs = env.reset()
i = 0
info = None
while True:
    print(i)
    if i >= 3:
        break
    with torch.no_grad():
        value, action, _, recurrent_hidden_states, logits = actor_critic.act(
            obs, recurrent_hidden_states, masks, deterministic=True)

    x = 0 if info is None else info[0]['x']
    max_x = 0 if info is None else info[0]['max_x']
    pct = 0 if info is None else info[0]['max_x']/info[0]['lvl_max_x']*100
    rew = 0 if info is None else info[0]['sum_r']
    t = i
    a = action.item()
    tgt_layer = {'video': 5, 'audio': 5}
    f = gen_eval_vid_frame(
        actor_critic, env_state, x, max_x, pct, rew, t, a, logits, obs, tgt_layer)
    display(Image.fromarray(f, 'RGB'))
    

    # Obser reward and next obs
    obs, reward, done, info = env.step(action)
    masks.fill_(0.0 if done else 1.0)
    
    if done:
        print(f"max_x {info[0]['max_x']}, stage len {info[0]['screen_x_end']}")
        break
    i += 1

0
dict_keys(['video', 'mel_s', 'audio'])
torch.Size([1, 1, 256, 92])


RuntimeError: Padding size should be less than the corresponding input dimension, but got: padding (512, 512) at dimension 2 of input (1,.,.) = 
 Columns 1 to 9  0.0593  0.0593  0.0593  0.0593  0.0593  0.0593  0.0601  0.0611  0.0621
 -0.0000 -0.0000 -0.0000 -0.0000 -0.0000 -0.0000 -0.0000 -0.0000 -0.0000
  0.0406  0.0402  0.0388  0.0370  0.0349  0.0328  0.0314  0.0306  0.0305
  0.0292  0.0287  0.0274  0.0255  0.0234  0.0214  0.0198  0.0188  0.0185
  0.0471  0.0455  0.0411  0.0352  0.0291  0.0234  0.0188  0.0155  0.0137
  0.0808  0.0792  0.0749  0.0691  0.0631  0.0575  0.0535  0.0508  0.0497
  0.0764  0.0713  0.0593  0.0455  0.0330  0.0223  0.0139  0.0076  0.0037
  0.1956  0.1752  0.1437  0.1180  0.0983  0.0830  0.0722  0.0644  0.0592
  0.1841  0.1520  0.1130  0.0849  0.0642  0.0486  0.0371  0.0284  0.0223
  0.1874  0.1646  0.1310  0.1039  0.0827  0.0660  0.0533  0.0431  0.0350
  0.2011  0.1788  0.1458  0.1193  0.0986  0.0821  0.0700  0.0605  0.0530
  0.0967  0.0938  0.0861  0.0753  0.0